In [4]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, Iterator, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import bits
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import sorting
import stack
import strings
import trees

for m in (
    arrays,
    bits,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    sorting,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from mathematics import prime_numbers as primes
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(edgeitems=5, precision=3, linewidth=300)

In [ ]:
def longest_balanced_parenthesis_substring(s: str) -> int:
    # There is absolutely no need for pushing stack, given that there is only one and
    # the same token pushed and we only need the depth of stack, so this solution
    # uses a counter for this. The idea is to loop over from left (and also from right)
    # while keeping the counter of open parentheses. Each time time counter falls
    # below 0, reset the starting position on the string and the counter to 0.

    def longest(s: Iterable, oc: str = "()") -> int:
        op, cl = oc
        open_count = 0
        mx = 0
        start = -1
        for i, c in enumerate(s):
            if c == op:
                open_count += 1
            elif c == cl:
                open_count -= 1
                if open_count == 0:
                    mx = max(mx, i - start)
                elif open_count < 0:
                    start = i
                    open_count = 0
        return mx

    return max(longest(s), longest(reversed(s), oc=")("))

In [20]:
longest_balanced_parenthesis_substring("(()(")

2

In [21]:
longest_balanced_parenthesis_substring("()(())(")

6